In [ ]:
install.packages("mvProbit")
library(mvProbit)
options( digits = 6 )

In [ ]:
#read data created by "data processing"
d <- read.csv("D:/Processed Data/NRW_ownership_input.csv")

In [ ]:
#enter "mvProbit-world". Throw out all columns not used
#WITHOUT dummies for Raumtyp_11.0, alter_gr_4, P_BIL_1, HP_SEX_1, taet_1, oek_status_3, hhgr_gr2_1
d_mvProbit <- subset(d, select = c(cbike, ebike, elevar, Raumtyp_12, Raumtyp_21, Raumtyp_22,  alter_gr_1, alter_gr_2, alter_gr_3, alter_gr_5, alter_gr_6, alter_gr_7, alter_gr_8, P_BIL_1, P_BIL_2, P_BIL_3, P_BIL_5, P_BIL_6, HP_SEX_2, taet_2, taet_3, taet_4, taet_5, oek_status_1, oek_status_2, oek_status_4, oek_status_5, hhgr_gr2_2, hhgr_gr2_3, hhgr_gr2_4))

#add column "constant" (always 1) to estimate ASC
nObs <- nrow(d_mvProbit)
const <- rep(1, nObs)
d_mvProbit <- cbind(d_mvProbit, const)

#test with sample of 1000 observations
#d_mvProbit_1000 <- d_mvProbit[sample(nrow(d_mvProbit), 1000),]

In [ ]:
#define starting values for betas; one beta more than parameters (for constant which is not called explicitly in mvProbit function)
#starting values from a past estimation
beta <- cbind(c(1.2,-15.5,0,0,0,0.1,-0.4,-0.2,-0.2,-0.3,-0.4,-1,-0.2,-0.2,0,0.1,0.2,-0.1,0.1,-0.3,-0.2,-0.2,-0.2,-0.1,0.2,0.2,0.2,0.3,0.4),
             c(-1.9,-0.5,0,0,0,-1.2,-1.6,-0.4,0.3,0.5,0.4,0.1,-0.2,0.2,0.2,0,0,0,0.1,0.1,0.2,0,-0.2,-0.2,0,0.1,0.3,0.1,0.1))

#define starting values for covariance of error terms
#starting values from a past estimation
sigma <- cbind(c(1,-0.27),
              c(-0.27,1))

In [ ]:
#mvProbit estimation
estResult <- mvProbit(cbind(cbike, ebike)~elevar+Raumtyp_12+Raumtyp_21+Raumtyp_22+alter_gr_1+alter_gr_2+alter_gr_3+alter_gr_5+alter_gr_6+alter_gr_7+alter_gr_8+P_BIL_1+P_BIL_2+P_BIL_3+P_BIL_5+P_BIL_6+HP_SEX_2+taet_2+taet_3+taet_4+taet_5+oek_status_1+oek_status_2+oek_status_4+oek_status_5+hhgr_gr2_2+hhgr_gr2_3+hhgr_gr2_4,
                     start = c(beta),
                      startSigma = sigma,
                      data = d_mvProbit, iterlim = 4, nGHK = 50,
                     method = "BHHH")

In [ ]:
summary(estResult)

In [ ]:
#Calculate null-log-likelihood

#data
d_mvProbit_0 <- d_mvProbit

# model coefficients
beta_0 <- cbind(c(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
               c(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))

# covariance matrix of error terms
sigma_0 <- cbind(c(1,0),
              c(0,1))


# log likelihood values
logLikVal <- mvProbitLogLik( cbind(cbike, ebike)~elevar+Raumtyp_12+Raumtyp_21+Raumtyp_22+alter_gr_1+alter_gr_2+alter_gr_3+alter_gr_5+alter_gr_6+alter_gr_7+alter_gr_8+P_BIL_1+P_BIL_2+P_BIL_3+P_BIL_5+P_BIL_6+HP_SEX_2+taet_2+taet_3+taet_4+taet_5+oek_status_1+oek_status_2+oek_status_4+oek_status_5+hhgr_gr2_2+hhgr_gr2_3+hhgr_gr2_4, 
   coef = c( beta_0 ), sigma = sigma_0, data = d_mvProbit_0 )

sum(logLikVal)

In [ ]:
#hessian(estResult)
#vcov( estResult, eigentol=1e-12)